In [1]:
pip install sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
import numpy as np

from transformers import AutoModel, AutoTokenizer
from sentence_transformers import SentenceTransformer  # Import from the correct library
from scipy.spatial.distance import cosine

from topsis import topsis


In [8]:
models = ["BAAI/bge-m3", "intfloat/e5-large-v2", 
          "sentence-transformers/all-MiniLM-L6-v2", "sentence-transformers/all-mpnet-base-v2"]

tokenizers = [AutoTokenizer.from_pretrained(model) for model in models]
models = [AutoModel.from_pretrained(model) for model in models]


In [9]:
sentence1 = "This is a sentence about machine learning."
sentence2 = "This sentence discusses artificial intelligence."


In [10]:
embeddings = []
for tokenizer, model in zip(tokenizers, models):
    input_ids = tokenizer.encode(sentence1, sentence2, return_tensors="pt")
    with torch.no_grad():
        output = model(input_ids)
    embeddings.append(output.pooler_output)

In [11]:
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
similarities = []
for i in range(len(embeddings) - 1):
    sim = 1 - torch.cosine_similarity(embeddings[i], embeddings[i+1], dim=-1)
    similarities.append(sim)
if len(embeddings) > 1:
    similarities = 1 - torch.cosine_similarity(embeddings[0], torch.cat(embeddings[1:]), dim=-1)

RuntimeError: The size of tensor a (1024) must match the size of tensor b (384) at non-singleton dimension 1

In [12]:
criteria = ["accuracy", "speed", "memory_usage"]  # Replace with your actual criteria
weights = [0.4, 0.3, 0.3]  # Adjust weights as needed
ideal_best = [1, 1, 1]  # Assuming higher values are better for all criteria
ideal_worst = [0, 0, 0]

In [13]:
decision_matrix = [
    [model_name] + list(sim) for model_name, sim in zip(models[1:], similarities)
]
results = topsis(decision_matrix, weights, ideal_best, ideal_worst)
best_model_index = results["rank"].index(1)
best_model = models[best_model_index + 1]

TypeError: 'module' object is not callable

In [ ]:
print("Best model:", best_model)